<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/khaiii_%ED%85%8C%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/kakao/khaiii.git

Cloning into 'khaiii'...
remote: Enumerating objects: 877, done.
remote: Total 877 (delta 0), reused 0 (delta 0), pack-reused 877
Receiving objects: 100% (877/877), 30.21 MiB | 28.51 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [0]:
!cd khaiii
!mkdir build
!cd build

In [0]:
import os 
os.chdir('/content/khaiii/')

In [0]:
!mkdir build

In [0]:
os.chdir('/content/khaiii/build/')

In [7]:
!cmake ..

-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test fma_compiles
-- Performing Test fma_compiles - Success
-- Performing Test fma_runs
-- Performing Test fma_runs - Success
-- [khaiii] fused multiply add option enab

In [8]:
!make all

Scanning dependencies of target obj_khaiii
[  3%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Config.cpp.o
[  7%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Embed.cpp.o
[ 11%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/ErrPatch.cpp.o
[ 15%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/KhaiiiImpl.cpp.o
[ 19%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Morph.cpp.o
[ 23%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Preanal.cpp.o
[ 26%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Resource.cpp.o
[ 30%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Restore.cpp.o
[ 34%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Sentence.cpp.o
[ 38%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Tagger.cpp.o
[ 42%] Building CXX object CMakeFiles/obj_khaiii.dir/src/main/cpp/khaiii/Trie.cpp.o
[ 4

In [9]:
!make resource

Scanning dependencies of target resource
INFO:root:config: {
    "cutoff": 1,
    "embed_dim": 35,
    "hidden_dim": 320,
    "model_id": "munjong.cut1.win4.sdo0.1.emb35.lr0.001.lrd0.9.bs500",
    "rsc_src": "../rsc/src",
    "window": 4
}
INFO:root:vocab.in: 5621 entries, 0 cutoff
INFO:root:vocab.out: 500 entries, 0 cutoff
INFO:root:restore.dic: 4303 entries
INFO:root:preanal.auto
INFO:root:preanal.manual
INFO:root:trie saved: /content/khaiii/build/share/khaiii/preanal.tri
INFO:root:total nodes: 120812
INFO:root:expected size: 1932992
INFO:root:value saved: /content/khaiii/build/share/khaiii/preanal.val
INFO:root:total entries: 83373
INFO:root:expected size: 593730
INFO:root:restore.key: 17212
INFO:root:restore.val: 68848
INFO:root:restore.one: 503
INFO:root:base.errpatch.auto
INFO:root:base.errpatch.manual
INFO:root:trie saved: /content/khaiii/build/share/khaiii/errpatch.tri
INFO:root:total nodes: 2067
INFO:root:expected size: 33072
INFO:root:length saved: /content/khaiii/build/share

In [10]:
os.chdir('/content/khaiii/build/')
!./bin/khaiii --rsc-dir=./share/khaiii

[2019-11-25 20:54:13.783] [Resource] [info] NN model loaded
[2019-11-25 20:54:13.783] [Preanal] [info] preanal dictionary opened
[2019-11-25 20:54:13.783] [ErrPatch] [info] errpatch dictionary opened
[2019-11-25 20:54:13.783] [Restore] [info] restore dictionary opened
[2019-11-25 20:54:13.783] [Resource] [info] PoS tagger opened
^C


In [11]:
!ctest

Test project /content/khaiii/build
    Start 1: test_khaiii
1/1 Test #1: test_khaiii ......................   Passed    0.02 sec

100% tests passed, 0 tests failed out of 1

Total Test time (real) =   0.03 sec


In [12]:
!make package_python

Scanning dependencies of target package_python
Run CPack packaging tool for source...
CPack: Create package using ZIP
CPack: Install projects
CPack: - Install directory: /content/khaiii
CPack: Create package
CPack: - package: /content/khaiii/build/khaiii-0.4.zip generated.
Built target package_python


In [13]:
os.chdir("/content/khaiii/build/package_python/")
!pip install .

Processing /content/khaiii/build/package_python
  Created wheel for khaiii: filename=khaiii-0.4-cp36-none-any.whl size=22891492 sha256=c38d244c73814bc512f36e900003c25d17b337bb64549a36a23d733903264e8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-5tvcrsyw/wheels/b0/03/83/fb129110ddf28d31298b0731cd182d7d0f094935ae9bf8d4bc
Successfully built khaiii


In [0]:
os.chdir("/content/")
!mkdir simmilar_work
os.chdir("/content/simmilar_work/")

In [0]:
os.chdir("/content/simmilar_work/")
import numpy as np
import pandas as pd

In [0]:
!ls

In [18]:
# 파일 임포팅 파트
from google.colab import files
uploaded = files.upload()

Saving 191101_테스트셋.xlsx to 191101_테스트셋.xlsx


In [19]:
# 파일 DF로 읽기

RStext = pd.read_excel("191101_테스트셋.xlsx", encoding = "UTF-8")
del RStext["Unnamed: 0"]
RStext.columns

Index(['과제수행년도', '부처명', '부처명(다부처포함)', '사업명', '과제고유번호', '보안과제여부', '신규계속구분',
       '과제명-국문', '과학기술표준분류1-대', '과학기술표준분류가중치1', '총연구비_합계(원)', '요약문_연구목표',
       '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드', '요약문_영문키워드', '과제관리기관', '과제명-국문_길이',
       '요약문_연구목표_길이', '요약문_연구내용_길이', '요약문_기대효과_길이', '요약문_한글키워드_길이',
       '요약문_영문키워드_길이'],
      dtype='object')

In [0]:
# 텍스트 정리
RStext['과제명-국문_정리'] = RStext['과제명-국문'].str.replace("\t|\n|\r","")
RStext['요약문_연구목표_정리'] = RStext['요약문_연구목표'].str.replace("\t|\n|\r","")
RStext['요약문_연구내용_정리'] = RStext['요약문_연구내용'].str.replace("\t|\n|\r","")
RStext['요약문_기대효과_정리'] = RStext['요약문_기대효과'].str.replace("\t|\n|\r","")
RStext['요약문_한글키워드_정리'] = RStext['요약문_한글키워드'].str.replace("\t|\n|\r","").str.replace(",",", ").str.replace("  "," ")
RStext['요약문_영문키워드_정리'] = RStext['요약문_영문키워드'].str.replace("\t|\n|\r","").str.replace(",",", ").str.replace("  "," ")

In [21]:
# 통합텍스트 셀 생성
RStext["전체 텍스트"] = RStext['과제명-국문_정리'] + " " + RStext['요약문_연구목표_정리'] + " " + RStext['요약문_연구내용_정리'] + " " + RStext['요약문_기대효과_정리'] + " " + RStext['요약문_한글키워드_정리'] + " " + RStext['요약문_영문키워드_정리']
RStext["전체 텍스트_기본"] = RStext["전체 텍스트"].str.findall('[a-z|A-Z|가-힣|&|.-.]+')
for i in range(len(RStext["전체 텍스트_기본"])):
  RStext["전체 텍스트_기본"][i] = " ".join(RStext["전체 텍스트_기본"][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
import numpy as np
import pandas as pd
import re
from random import sample
from khaiii import KhaiiiApi
import time
api = KhaiiiApi()

In [0]:
Npercnt_list = [90, 80, 70, 60, 50, 40]
ch_keyword = "더미"

subj_for_test = pd.DataFrame()
subj_for_test["전체 텍스트"] = RStext["전체 텍스트"].str.split(" ")
subj_for_test["제목 길이"] = 0
subj_for_test["목표 길이"] = 0
subj_for_test["내용 길이"] = 0
subj_for_test["효과 길이"] = 0
subj_for_test["한키 길이"] = 0
subj_for_test["영키 길이"] = 0
subj_for_test["제목 추출"] = ''
subj_for_test["목표 추출"] = ''
subj_for_test["내용 추출"] = ''
subj_for_test["효과 추출"] = ''
subj_for_test["한키 추출"] = ''
subj_for_test["영키 추출"] = ''
subj_for_test["과제 번호"] = RStext["과제고유번호"]
subj_for_test["기대 유사"] = 0

In [24]:
print(subj_for_test["전체 텍스트"][0])

['바이러스', '현장', '검출용', '초소형화', '펜타입', '표면', '플라즈몬', '공명', '시스템', '개발', '○', '본', '제안기술은', 'SPR', '시스템', '기술,', '광학', '집적', '기술,', '소프트웨어', '기술,', '바이오컨쥬게이셔', '기술을', '바탕으로', '하여,', '바이러스', '현장', '진단용', '초소형화', '펜타입', 'SPR', '시스템을', '세계최초로', '개발하는', '것을', '주요', '골자로', '함.○', '본', '제안기술은', 'Kretchmann', 'configuration', '기반의', 'SPR', '센싱', '방법을', '주요', '적용', '기술로서', '활용하지만,', '추가적으로', '최근', '연구가', '활발히', '진행되고', '있는', '나노', '플라즈모닉', '센싱', '기반으로', '개발', '범위를', '확장하여', '학술적으로도', '깊이', '있는', '신규의', '센싱', '기법을', '개발하고자', '함.', '○', '초소형화', '펜타입', 'SPR', '시스템', '설계/제작', '기술', '연구-', 'Kretchmann', 'configuration', '기반', '광학', '미세', '집적', '기술', '연구-', '나노플라즈모닉', '기반', '광학', '미세', '집적', '기술', '연구-', '일회용', '칩', '탈부착', '기술', '및', '액체유입', '차단', '기술', '연구-', '센서', '분해능:', '1', 'x', '10-7', 'RIU', '(refractive', 'index', 'unit)-', 'SPR', '광학모듈', '크기:', '15', 'x', '15', 'x', '50', 'mm', '(W', 'x', 'D', 'x', 'H)-', '무선,', '저전력', '시스템', '집적', '기술', '개발○', '일회용', '칩', '설계/제작', '기술', '및', '바이러스', '측정', '기술

In [0]:
subj_for_test_total = pd.DataFrame()

In [37]:
for echo in Npercnt_list:
  Npercnt = echo
  for Tround in range(20):  
    for i in range(len(subj_for_test)):
      # 텍스트와 길이 정의, 본문은 전체 텍스트를 띄어쓰기 형태로 붙였음. 
      sub_text0 = str(" " + " ".join(subj_for_test["전체 텍스트"][i])+" ") 
      sub_text = str(" " + " ".join(subj_for_test["전체 텍스트"][i])+" ") 
      subj_for_test["제목 길이"][i] = len(RStext["과제명-국문_정리"][i])+1
      subj_for_test["목표 길이"][i] = int(subj_for_test["제목 길이"][i]) + len(RStext["요약문_연구목표_정리"][i])+1
      subj_for_test["내용 길이"][i] = int(subj_for_test["목표 길이"][i]) + len(RStext["요약문_연구내용_정리"][i])+1
      subj_for_test["효과 길이"][i] = int(subj_for_test["내용 길이"][i]) + len(RStext["요약문_기대효과_정리"][i])+1
      subj_for_test["한키 길이"][i] = int(subj_for_test["효과 길이"][i]) + len(RStext["요약문_한글키워드_정리"][i])+1
      subj_for_test["영키 길이"][i] = int(subj_for_test["한키 길이"][i]) + len(RStext["요약문_영문키워드_정리"][i])+1
      # 형태소 분석, 아직 문서 하나
      test = []
      for word in api.analyze(RStext["전체 텍스트"][i]):
        test.append(str(word))
        test_str = re.sub('[A-Za-z가-힣]*\t|,'," "," ".join(test))
        test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
        test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
        test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
        test_set = list(set(test_str))
        test_str = str(" "+" ".join(test_str)+" ")
      subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
      subj_count["위치"] = ""
      subj_count["위치변화"] = ""
      subj_count["변경수"] = 0
      subj_count["변경대상"] = ""
      subj_count["대체단어"] = ch_keyword
      subj_count["더미글자"] = "뷁"
      for k in range(len(test_set)):
        subj_count["키워드"][k] = test_set[k]
        subj_count["빈도"][k] = len(re.compile(str(" "+test_set[k]+" ")).findall(test_str))
        subj_count["위치"][k] = [m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]
        subj_count["위치변화"][k] = sample(subj_count["위치"][k],len(subj_count["위치"][k]))
      subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
      subj_count["대체단어길이"] = subj_count["키워드"].str.len()-2
      subj_count["대체용어"] = subj_count["대체단어"] + (subj_count["더미글자"] * subj_count["대체단어길이"])
      Keycount = int(round(sum(subj_count["빈도"])/100*(100-Npercnt),0))
      for j in range(len(subj_count)):
        subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
        if Keycount < 0:
          subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
          subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
          break
        subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
      for l in range(len(subj_count)):
        if len(subj_count["변경대상"][l]) == 0:
          continue
        else:
          for ll in range(len(subj_count["변경대상"][l])):
            sub_text = "".join((sub_text[:subj_count["변경대상"][l][ll]],subj_count["대체용어"][l],sub_text[subj_count["변경대상"][l][ll]+len(subj_count["키워드"][l]):]))
      subj_for_test["제목 추출"][i] = sub_text[1:subj_for_test["제목 길이"][i]]
      subj_for_test["목표 추출"][i] = sub_text[subj_for_test["제목 길이"][i]+1:subj_for_test["목표 길이"][i]]
      subj_for_test["내용 추출"][i] = sub_text[subj_for_test["목표 길이"][i]+1:subj_for_test["내용 길이"][i]]
      subj_for_test["효과 추출"][i] = sub_text[subj_for_test["내용 길이"][i]+1:subj_for_test["효과 길이"][i]]
      subj_for_test["한키 추출"][i] = sub_text[subj_for_test["효과 길이"][i]+1:subj_for_test["한키 길이"][i]]
      subj_for_test["영키 추출"][i] = sub_text[subj_for_test["한키 길이"][i]+1:subj_for_test["영키 길이"][i]]
    subj_for_test["기대 유사"] = Npercnt
    subj_for_test_total = pd.concat([subj_for_test_total,subj_for_test])

subj_for_test_total["제목 추출"] = subj_for_test_total["제목 추출"].str.replace("뷁","")
subj_for_test_total["목표 추출"] = subj_for_test_total["목표 추출"].str.replace("뷁","")
subj_for_test_total["내용 추출"] = subj_for_test_total["내용 추출"].str.replace("뷁","")
subj_for_test_total["효과 추출"] = subj_for_test_total["효과 추출"].str.replace("뷁","")
subj_for_test_total["한키 추출"] = subj_for_test_total["한키 추출"].str.replace("뷁","")
subj_for_test_total["영키 추출"] = subj_for_test_total["영키 추출"].str.replace("뷁","")



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-p

In [39]:
len(subj_for_test_total)

3840

In [0]:
subj_for_test_total.to_excel("subj_for_test_total.xlsx")

In [0]:
files.download("subj_for_test_total.xlsx")

In [28]:
test = []
start = time.time()
for word in api.analyze(RStext["전체 텍스트"][i]):
  test.append(str(word))
  np.random.randint(0,1000000,1000)
print("time :", time.time() - start)

time : 0.04223060607910156


In [29]:
print(subj_for_test["전체 텍스트"][i])
print(sub_text0)
print("과제1: ", RStext['과제명-국문_정리'][i])
print("과제2: ", subj_for_test["제목 추출"][i])
print("목표1: ", RStext['요약문_연구목표_정리'][i])
print("목표2: ", subj_for_test["목표 추출"][i])
print("내용1: ", RStext['요약문_연구내용_정리'][i])
print("내용2: ", subj_for_test["내용 추출"][i])
print("효과1: ", RStext['요약문_기대효과_정리'][i])
print("효과2: ", subj_for_test["효과 추출"][i])
print("한키1: ", RStext['요약문_한글키워드_정리'][i])
print("한키2: ", subj_for_test["한키 추출"][i])
print("영키1: ", RStext['요약문_영문키워드_정리'][i])
print("영키2: ", subj_for_test["영키 추출"][i])

['반성을', '촉발하는', '사건:', '지각과', '기분을', '중심으로', '이', '연구는', '대상을', '범주화하는', '통상적', '지각', '속에선', '현시', '불가능한', '것들이', '부지불식간에', '감각적으로', '현시되는', '현상에', '주목하며,', '이를', '[사건]이라', '부르도록', '한다.', '‘사건’은', '인간을', '규정하는', '가장', '구체적', '실재인', '[지각]과', '[기분],', '[사유]의', '상관관계를', '드러낸다는', '점에서', '중요하다.', '사건은', '보이지', '않던', '것이', '보이게', '된', '것이기에', '‘확장된', '지각’이며,', '이', '확장된', '지각을', '가능하게', '하는', '근본', '토대는', '곧', '‘기분’이다.', '그리고', '기분에', '젖어', '지각이', '확장되는', '사건', '속에선', '미지의', '것들에', '대한', '반성이', '시작되므로,', '‘사유의', '확장’이', '일어난다.', '본', '연구는', '사건을', '키워드로', '삼아', '인간', '정신의', '핵심요소들에', '대한', '새로운', '지형도를', '그려봄으로써', '인간에', '대한', '보다', '깊은', '이해에', '접근할', '것이다.', '‘사건’은', '프랑스', '현대', '철학을', '관통하는', '주제', '중', '하나이다.', '그러나', '여러', '프랑스', '철학자들의', '사건', '개념은', '공통점과', '차이점이', '있음에도', '아직', '종합적으로', '사유되지는', '못하고', '있다.', '이', '연구는', '들뢰즈의', '사건', '개념,', '리오타르의', '사건', '개념,', '메를로-퐁티의', '지각론과', '예술론', '등을', '검토하면서,', '사건', '개념을', '인간', '인식', '능력', '및', '예술과의', '관련', '하에서', '재정립하도록', '한다.', '이를',

In [38]:
subj_for_test.head(100)

,전체 텍스트,제목 길이,목표 길이,내용 길이,효과 길이,한키 길이,영키 길이,제목 추출,목표 추출,내용 추출,효과 추출,한키 추출,영키 추출,과제 번호,기대 유사
0,"[바이러스, 현장, 검출용, 초소형화, 펜타입, 표면, 플라즈몬, 공명, 시스템, ...",39,311,1010,1551,1613,1741,더미뷁뷁 현장 검출용 더미뷁뷁 펜타입 표면 더미뷁몬 공명 시스템 개발,"○ 본 더미더미은 SPR 더미뷁 더미, 광학 집적 기술, 더미뷁뷁뷁 더미, 바이오컨...",○ 초소형화 더미뷁 SPR 더미뷁 더미/제작 기술 더미- 더미뷁뷁뷁뷁뷁뷁뷁뷁 더미뷁...,○ 더미기술적 더미더미와 더미효과- 본 연구에서 더미하는 새로운 더미의 「더미뷁 무...,"더미 더미뷁뷁 공명, 스마트폰 이용 더미기기, 더미뷁뷁 더미, 나더미라즈모닉 센서,...","surfaceplasmonresonance, 더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁...",1345248282,40
1,"[휴대용, 복합, 재머용, 재머, 신호발생기, 개발, 본, 연구, 개발의, 주, 목...",23,181,904,1351,1388,1440,더미용 더미 더미용 재머 신호더미뷁 개발,본 연구 개발의 주 목표는 더미더미기의 C2 데이터링크 및 다대역 위성더미수신기에 ...,본 더미 개발에서는 더미용이 가능한 더미 더미 더미뷁를 정해진 규격(더미(210x2...,최근 더미 비행기의 더미용을 더미로 더미한 상업용 용도가 증가하고 있다. 이와 더불...,"더미, 위상 조절기, 더미역, 더미뷁, 초고주파 더미뷁로, 안테나","Jammer, PhaseShifter, Wideband, CMOS, 더미뷁뷁, An...",1711045805,40
2,"[Listeria, monocytogenes, 시그마B, 전사인자가, 막, 유래, ...",68,408,1043,1271,1331,1432,Listeria monocytogenes 시그마B 전사인자가 막 더미 나노 더미뷁 ...,Listeria monocytogenes wild-type 균주와 ΔsigB 돌연변...,1. L. monocytogenes wild-type 균주와 ΔsigB 돌연변이 균...,○ 세균 병원성 더미의 새로운 더미 개척L. monocytogenes가 숙주 내 감...,"더미테리아 모노사이토더미뷁, 시그마 전사인자, 막 더미 나노 더미뷁, 더더미 온톨로...","더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁, 더미뷁뷁뷁뷁, 더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁...",1345248236,40
3,"[제품의, 해외, 등록과, 수출, 활성화, 및, 호르몬, 제제의, 버팔로, 품종에,...",44,142,866,1467,1492,1560,제품의 해외 더미과 수출 활성화 및 호르몬 더미의 버팔로 품종에 대한 시험더미,동방 제품의 더미더미 및 수출활성화와 더미뷁 더미의 버팔로 품종에 대한 시험더미- ...,"● 더미조사 및 제품 더미: 필리핀과 더미뷁뷁의 축산업 현황 및 의약품 사용현황, ...",○ 과학적 더미 자료 확보를 통한 동물용의약품 수출 확대 - 해외현지 더미시험 ...,"수출, 제품더미, 더미평가, 호르몬, 버팔로","더미뷁뷁뷁뷁, 더미뷁뷁뷁뷁뷁 더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁, 더미뷁뷁뷁뷁뷁뷁 더미뷁뷁뷁뷁뷁...",1545016066,40
4,"[5G, 이동, 단말을, 위한, 적응형, 능동, RF, 시스템, 설계에, 관한, 연...",37,520,1228,1812,1878,2046,5G 더미 단말을 위한 적응형 더미 RF 더미뷁 설계에 관한 연구,본 제안 과제는 5G 고속 데이터 더미에 적합한 더미형 더미 RF 더미뷁을 새로운 ...,"본 더미에서는 기존의 통신 더미에서 수행하였던 프로세싱 더미뷁 신호의 인지, 제어 ...",본 연구 과제는 새로운 RF FE의 더미뷁뷁을 제안하는 것과 더미에 현재 활발하게 ...,"더미형 더미 RF 시스템, 아날로그 프로더미, 채널 인지, 코딩 빔 안테나, 5G ...","더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁, 더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁, ...",1345248484,40
5,"[통합, 내진성능평가방법과, 댐퍼, 이용한, 비정형건물, 내진성능개선방법, 개발, ...",38,516,1546,2011,2064,2210,통합 더미더미더미뷁뷁과 더미 이용한 비정형더미 더미성능더미뷁뷁 개발,본 연구의 목표는 기존 더미 비정형 건물의 더미더미을 평가하기 위하여 1차와 2차 ...,본 더미의 더미 주제는 더미적 더미더미평가법과 댐퍼 이용한 비정형더미 더미더미개선방...,(1) 국내의 더미더미을 더미한 더미뷁뷁 응답스펙트럼 더미과 지반운동 라이브더미 더...,"더미성능더미, 더미성능더미, 마찰더미, 붕괴, 동적해석, 지진위험도, 피해도, 취약...","더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁, 더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁...",1711030483,40
6,"[체공성이, 향상된, 드론, 설계를, 위한, 구조전지, 최근, 늘어나는, 에너지수요...",24,456,1204,1554,1601,1730,더미성이 더미된 더미 설계를 위한 구조더미,최근 늘어나는 에너지수요와 더미문제로 인하여 더미추진 항공기 더미이 각광받고 있지만...,본 연구는 더미뷁성 구조 더미를 더미하는 더미 추진 드론의 더미설계를 통하여 더미뷁...,"군사적 더미으로는 체공더미 증대를 통한 임무더미 확장이 가능하고, 산업적 더미에서는...","더미지지 구조더미, 더미뷁 구조더미 더미, 더미뷁성 더미뷁, 더미성 더미, 더미설계","LoadcarryingStructuralBattery, CompositeStruct...",1711045373,40
7,"[페로브스카이트용, 전하, 수송재료, 및, 박막공정, 개발, 고효율?고안정성, 페로...",27,410,1177,1652,1689,1767,페로브더미뷁뷁용 더미 더미더미 및 더미공정 더미,고더미?고안정성 더미뷁스카이트 더미더미을 위한 페로스브카이트 발광체용 유기재료와 정...,더미 수송층(HTL/ETL) 더미용 원천성 코어 기반의 신규 더미 합성과 데이터베이...,페로브스카이트 소재와 더미 관련 과학더미 분야의 더미 경쟁력 및 더미 제고를 기대할...,"페로더미뷁뷁뷁, 나노 결정, 더미 더미층, 용액 공정, 분사 공정","Perovskite, Nanocrystal, Carriertransportlayer...",1711057725,40
8,"[딥러닝을, 이용한, 다중템플릿, 기반, 자동, 영상, 분할, 기법, 고속, 처리,...",42,106,724,1030,1068,1127,딥러닝을 이용한 더미뷁뷁뷁 기반 자동 영상 분할 기법 고속 처리 기술 개발,딥러닝을 이용한 더미뷁뷁뷁 기반 자동 영상 분할 기법 더미 처리 기술 개발: 의료영...,딥러닝 (Deep 더미뷁뷁뷁뷁뷁)을 더미한 내용기반 영상질의 시스템 (Content...,"최근 의료더미뷁뷁뷁뷁에 대한 관심이 크게 증가하고 있으며, 이러한 의료영상빅데이터 ...","영상더미, 더미뷁뷁뷁, 딥러닝, 내용기반 영상질의 시스템, 빅데이터","ImageSegmentation, Multiatlas, 더미뷁뷁Learning, C...",1711039607,40
9,"[비암호화, 및, 암호화, 유전자, 변이, 프로파일링을, 이용한, 간암진화, 모델링...",51,127,858,1291,1330,1449,더미뷁뷁 및 암호화 유전자 변이 더미뷁뷁뷁을 이용한 더미진화 더미뷁 및 암화 더미더...,"더미더미 단계별 멀티오믹스의 통합분석을 이용하여 간암진화더미을 구축하고, 이를 기반...","1차년도-간암의 더미 더미별 시료 및 임상더미 더미 ○ 더미 단계 시료인 간경변, ...",○ 본 더미 더미 더미되는 더미 발생 더미별 멀티오믹스 더미뷁 프로파일은 세계적으로...,"간암, 더미뷁뷁뷁뷁뷁뷁, 암진화, 약물더미뷁, 비암호화, 더미뷁뷁수성","더미뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁뷁 carcinoma, multi-omics data, ca...",1711069872,40


In [31]:
sub_text
[m.start() for m in re.finditer(subj_count["키워드"][k],sub_text)]

[494]

In [32]:
subj_count.head()

,키워드,빈도,위치,위치변화,변경수,변경대상,대체단어,더미글자,대체단어길이,대체용어
0,사건,22.0,"[10, 98, 114, 179, 271, 322, 394, 443, 499, 51...","[1146, 795, 114, 1212, 271, 658, 1404, 1111, 3...",14,"[1146, 795, 114, 1212, 271, 658, 1404, 1111, 3...",더미,뷁,0,더미
1,지각,12.0,"[14, 47, 140, 209, 222, 262, 527, 698, 723, 76...","[14, 209, 527, 47, 698, 762, 952, 786, 140, 22...",0,[],더미,뷁,0,더미
2,확장,11.0,"[205, 218, 266, 305, 634, 731, 790, 1071, 1275...","[731, 1275, 218, 305, 205, 1283, 1071, 1424, 7...",8,"[731, 1275, 218, 305, 205, 1283, 1071, 1424]",더미,뷁,0,더미
3,예술,9.0,"[532, 564, 650, 907, 1220, 1298, 1313, 1340, 1...","[1220, 907, 564, 650, 1428, 1340, 532, 1382, 1...",0,[],더미,뷁,0,더미
4,개념,7.0,"[446, 502, 515, 549, 1149, 1215, 1256]","[446, 1215, 1256, 1149, 515, 502, 549]",6,"[446, 1215, 1256, 1149, 515, 502]",더미,뷁,0,더미


In [0]:
# # 
# test_str = re.sub('[A-Za-z가-힣]*\t'," "," ".join(test))
# test_str = re.compile('[A-Za-z가-힣]*/NNG|[A-Za-z가-힣]*/SL|[A-Za-z가-힣]*/NNP').findall(test_str)
# test_str = re.sub('/NNG|/SL|/NNP',""," ".join(test_str))
# test_str = re.compile('[A-Za-z가-힣][A-Za-z가-힣]+').findall(test_str)
# test_set = list(set(test_str))
# test_str = str(" "+" ".join(test_str)+" ")
# print(test_str)
# print(test_set)

In [0]:
# subj_count = pd.DataFrame(np.zeros([len(test_set),2]),columns = ["키워드","빈도"])
# subj_count["위치"] = ""
# subj_count["위치변화"] = ""
# subj_count["변경수"] = 0
# subj_count["변경대상"] = ""
# for i in range(len(test_set)):
#   subj_count["키워드"][i] = test_set[i]
#   subj_count["빈도"][i] = len(re.compile(str(" "+test_set[i]+" ")).findall(test_str))
#   subj_count["위치"][i] = list(np.where(np.array(test_str.split(" ")) == test_set[i])[0])
#   subj_count["위치변화"][i] = sample(subj_count["위치"][i],len(subj_count["위치"][i]))
# subj_count = subj_count.sort_values(by = ["빈도"],ascending=False).reset_index(drop=True)
# Npercnt = 10
# Keycount = int(round(sum(subj_count["빈도"])/100*Npercnt,0))
# for j in range(len(subj_count)):
#   subj_count["변경수"][j] = int(np.random.randint(0, high = int(subj_count["빈도"][j])+1,size = 1))*int(sample([0,1,1,1],1)[0])
#   Keycount -= subj_count["변경수"][j]
#   if Keycount < 0:
#     subj_count["변경수"][j] = subj_count["변경수"][j] - (0 - Keycount)
#     subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
#     break
#   subj_count["변경대상"][j] = list(subj_count["위치변화"][j][0:subj_count["변경수"][j]])
